In [ ]:
!pip install --extra-index-url https://pypi.netsquid.org netsquid

Looking in indexes: https://pypi.org/simple, https://pypi.netsquid.org


In [ ]:
import netsquid as ns
import netsquid.components.instructions as instr
from netsquid.components import QuantumChannel, ClassicalChannel
from netsquid.protocols import NodeProtocol
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.protocols import Protocol
from netsquid.nodes.network import Network
from netsquid.components.qprocessor import PhysicalInstruction
from scipy.stats import bernoulli
import numpy as np


In [ ]:
#free space
from scipy.linalg import expm
from scipy.special import i0, i1
from netsquid.components.models.qerrormodels import QuantumErrorModel
from netsquid.qubits import qubitapi as qapi, QFormalism
from netsquid.qubits import operators as ops
from netsquid.qubits.operators import Operator, Z, I
import netsquid.util.simtools as simtools
from netsquid.util.simlog import warn_deprecated
from numpy import random
from numpy.random import weibull
from itertools import combinations

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
node_distance = 4e-3
depolarization_rate = 1e7
dephasing_rate = 0.2

## **Test Free-Space Model**

In [ ]:
__all__ = [
    'FreeSpaceLossModel',
    'FixedSatelliteLossModel'
]

class FreeSpaceLossModel(QuantumErrorModel):
    """Model for photon loss on a free space channel

    Uses beam-wandering PDTC from [Vasylyev et al., PRL 108, 220501 (2012)] to
    sample the loss probability of the photon.

    Parameters
    ----------
    W0 : float
        Waist of the beam at the transmitter [m].
    rx_aperture : float
        Radius of the receiving telescope [m].
    Cn2 : float
        Index of refraction structure constant [m**(-2/3)].
    wavelength : float
        Wavelength of the radiation [m].
    Tatm : float
        Atmospheric transmittance (square of the transmission coefficient).
    sigmaPoint :
        Pointing error, standard deviation [rad].
    rng : :obj:`~numpy.random.RandomState` or None, optional
        Random number generator to use. If ``None`` then
        :obj:`~netsquid.util.simtools.get_random_state` is used.
    """
    def __init__(self, W0, rx_aperture, Cn2, wavelength, Tatm=1, sigmaPoint=0., rng=None):
        super().__init__()
        self.rng = rng if rng else simtools.get_random_state()
        self.W0 = W0
        self.rx_aperture = rx_aperture
        self.Cn2 = Cn2
        self.wavelength = wavelength
        self.sigmaPoint = sigmaPoint
        self.Tatm = Tatm
        self.required_properties = ['length']

    @property
    def rng(self):
        """ :obj:`~numpy.random.RandomState`: Random number generator."""
        return self.properties['rng']

    @rng.setter
    def rng(self, value):
        if not isinstance(value, np.random.RandomState):
            raise TypeError("{} is not a valid numpy RandomState".format(value))
        self.properties['rng'] = value

    @property
    def sigmaPoint(self):
        """ :float: pointing error at the transmitter [m]."""
        return self.properties['sigmaPoint']

    @sigmaPoint.setter
    def sigmaPoint(self, value):
        if value < 0:
            raise ValueError
        self.properties['sigmaPoint'] = value


    @property
    def Tatm(self):
        """ :float: atmosphere transmittance. """
        return self.properties['Tatm']

    @Tatm.setter
    def Tatm(self, value):
        if (value < 0) or (value > 1):
            raise ValueError
        self.properties['Tatm'] = value

    @property
    def W0(self):
        """float: beam waist at the transmitter [m]."""
        return self.properties['W0']

    @W0.setter
    def W0(self, value):
        if value < 0:
            raise ValueError
        self.properties['W0'] = value

    @property
    def rx_aperture(self):
        """float: radius of the receiving telescope [m]."""
        return self.properties['rx_aperture']

    @rx_aperture.setter
    def rx_aperture(self, value):
        if value < 0:
            raise ValueError
        self.properties['rx_aperture'] = value

    @property
    def Cn2(self):
        """float: index of refraction structure constant [m**(-2/3)]."""
        return self.properties['Cn2']

    @Cn2.setter
    def Cn2(self, value):
        if value < 0:
            raise ValueError
        self.properties['Cn2'] = value

    @property
    def wavelength(self):
        """float: wavelength of the radiation [m]."""
        return self.properties['wavelength']

    @wavelength.setter
    def wavelength(self, value):
        if value < 0:
            raise ValueError
        self.properties['wavelength'] = value

    def _compute_weibull_loss_model_parameters(self, length):
        """ Compute the parameters of the Weibull model

        This function calculates the parameters of the Weibull distribution
        lambda and R, described in equations (6) and (7) of [Vasylyev et al.,
        PRL 108, 220501 (2012)], and the maximal transmission T0, described in
        equation (5) of the same reference.

        It then converts these parameters to the parameters a and scaleL
        required by numpy to sample the Weibull distribution.

        Parameters
        ----------
        length: float
            Length of the channel.

        Returns
        -------
        tuple (float, float, float)
            The elements of the tuple are properties of the
            Weibull distribution. From left to right:
            - the 'shape' parameter
            - the 'scale' parameter
            - 'T0'
        """

        z = length*1e3
        W = self.W0*np.sqrt(1 + (z*self.wavelength/(np.pi*self.W0**2))**2)
        X = (self.rx_aperture/W)**2
        T0 = np.sqrt(1 - np.exp(-2*X))
        sigmaTurb = np.sqrt(1.919 * self.Cn2 * z**3 * (2*self.W0)**(-1./3.))
        sigma = np.sqrt( (self.sigmaPoint*z)**2 + sigmaTurb**2 )
        l = 8 * X * np.exp(-4*X) * i1(4*X) / (1 - np.exp(-4*X)*i0(4*X)) / np.log( 2*T0**2/(1-np.exp(-4*X)*i0(4*X)))
        R = self.rx_aperture * np.log( 2*T0**2/(1-np.exp(-4*X)*i0(4*X)) )**(-1./l)

        # define the parameters of the Weibull distribution
        a = 2/l
        scaleL = (2*(sigma/R)**2)**(l/2)

        return (a, scaleL, T0)

    def _sample_loss_probability(self, length):
        """ Sample the loss probability distribution

        This function samples the Weibull distribution to get the value of the
        channel transmittance, using the parameters calculated in the
        _compute_weibull_loss_model_parameter method.

        It uses the numpy.weibull method to implement the sampling of the
        probability distribution described in equation (9) of [Vasylyev et al.,
        PRL 108, 220501 (2012)].

        Parameters
        ----------
        length: float
            Length of the channel [km].

        Returns
        -------
        float
        """
        a, scaleL, T0 = self._compute_weibull_loss_model_parameters(length=length)

        # extract the value of the transmission coefficient
        x = weibull(a, 1)
        scaleX = scaleL * x
        T = T0*np.exp(-scaleX/2)
        # calculate the probability of losing the qubit
        prob_loss = 1 - self.Tatm * T**2
        return prob_loss

    def error_operation(self, qubits, delta_time=0, **kwargs):
        """Error operation to apply to qubits.

        Parameters
        ----------
        qubits : tuple of :obj:`~netsquid.qubits.qubit.Qubit`
            Qubits to apply noise to.
        delta_time : float, optional
            Time qubits have spent on a component [ns].

        """
        if 'channel' in kwargs:
            warn_deprecated("channel parameter is deprecated. "
                            "Pass length parameter directly instead.",
                            key="FreeSpaceLossModel.compute_model.channel")
            kwargs['length'] = kwargs['channel'].properties["length"]
            del kwargs['channel']


        for idx, qubit in enumerate(qubits):
            if qubit is None:
                continue
            prob_loss = self._sample_loss_probability(length=kwargs['length'])
            self.lose_qubit(qubits, idx, prob_loss, rng=self.properties['rng'])


class FixedSatelliteLossModel(FreeSpaceLossModel):
    """Model for photon loss on a satellite-to-ground static channel

    Uses beam-wandering PDTC from [Vasylyev et al., PRL 108, 220501 (2012)] to
    sample the loss probability of the photon.

    Parameters
    ----------
    txDiv : float
        Divergence of the beam sent from the satellite [rad].
    sigmaPoint :
        Pointing error of the satellite, standard deviation [rad].
    rx_aperture : float
        Radius of the receiving telescope [m].
    Cn2 : float
        Index of refraction structure constant [m**(-2/3)].
    wavelength : float
        Wavelength of the radiation [m].
    Tatm : float
        Atmospheric transmittance (square of the transmission coefficient).
    rng : :obj:`~numpy.random.RandomState` or None, optional
        Random number generator to use. If ``None`` then
        :obj:`~netsquid.util.simtools.get_random_state` is used.
    """
    def __init__(self, txDiv, sigmaPoint, rx_aperture, Cn2, wavelength, Tatm=1, rng=None):
        super().__init__(wavelength/(np.pi*txDiv),rx_aperture,Cn2,wavelength,Tatm,sigmaPoint,rng)
        self.txDiv = txDiv
        self.required_properties = ['length']

    @property
    def txDiv(self):
        """float: divergence of the beam at the transmitter (satellite) [m]."""
        return self.properties['txDiv']

    @txDiv.setter
    def txDiv(self, value):
        if value < 0:
            raise ValueError
        self.properties['txDiv'] = value

    def _compute_weibull_loss_model_parameters(self, length):
        """Compute the parameters of the Weibull model

        This function calculates the parameters of the Weibull distribution
        lambda and R, described in equations (6) and (7) of [Vasylyev et al.,
        PRL 108, 220501 (2012)], and the maximal transmission T0, described in
        equation (5) of the same reference.

        It then converts these parameters to the parameters a and scaleL
        required by numpy to sample the Weibull distribution.

        Parameters
        ----------
        length: float
            Length of the channel.

        Returns
        -------
        tuple (float, float, float)
            The elements of the tuple are properties of the
            Weibull distribution. From left to right:
            - the 'shape' parameter
            - the 'scale' parameter
            - 'T0'
        """

        # this function cannot be used for range values lower than 10 km
        if length <= 10:
            raise ValueError

        z = length*1e3
        W = self.txDiv * z
        X = (self.rx_aperture/W)**2
        T0 = np.sqrt(1 - np.exp(-2*X))
        sigmaTurb = np.sqrt(1.919 * self.Cn2 * 10e3**3 * (2*self.txDiv*(z-10e3))**(-1./3.))
        sigma = np.sqrt( (self.sigmaPoint*z)**2 + sigmaTurb**2 )
        l = 8 * X * np.exp(-4*X) * i1(4*X) / (1 - np.exp(-4*X)*i0(4*X)) / np.log( 2*T0**2/(1-np.exp(-4*X)*i0(4*X)))
        R = self.rx_aperture * np.log( 2*T0**2/(1-np.exp(-4*X)*i0(4*X)) )**(-1./l)

        # define the parameters of the Weibull distribution
        a = 2/l
        scaleL = (2*(sigma/R)**2)**(l/2)

        return (a, scaleL, T0)

In [ ]:
#### Free space channel parameter
W0 = 0.1
rx_aperture_freespace = 1
tatm = 1
Cn2_freespace = 1e-15 #atmosferic turbulence

In [ ]:
# Satllite to ground channel
distance_sat = 450
txDiv = 10e-6 #beam divergence
rx_aperture_sat = 1 #aperture of the receiving telescope
Cn2_sat = 1e-15
#pointing error
sigmaPoint = 0.5e-6

In [ ]:
#Light parameter
wavelength = 1550e-9 #telecom wavelength

In [ ]:
#Node parameters
init_time = 100 #time to create a qubit in ns

In [ ]:
Key = []

In [ ]:
simDuration = 10000       # simulation duration [ns]
init_time = 100           # time between qubit transmissions [ns]
num_qubits = int(simDuration / init_time)

# Dummy physical parameters (set these to match your system)
txDiv = 1.0
sigmaPoint = 1.0
rx_aperture_sat = 1.0
Cn2_sat = 1e-15
wavelength = 1550         # in nm
tatm = 1.0

In [ ]:
physical_instructions = [
    PhysicalInstruction(instr.INSTR_INIT, duration=init_time),
    PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_S, duration=1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_I, duration=1, parallel=True, topology=[0]),
    PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=1, parallel=True,topology=[0]),
    PhysicalInstruction(instr.INSTR_MEASURE_BELL, duration = 1, parallel=True),
    PhysicalInstruction(instr.INSTR_SWAP, duration = 1, parallel=True)
]

In [ ]:
#protocol
class SendProtocol(NodeProtocol):
  #Protocol performed by a node to send a BB84 qubit
  def __init__(self, othernode, node):
    super().__init__(node=node)
    self._othernode = othernode
  def run(self):
    if self.node.name[0:9] == "Satellite":
      mem = self.node.subcomponents["SatelliteMemory"]
    else:
      mem = self.node.subcomponents["Station"]
    while True:
      mem.reset()
      mem.execute_instruction(instr.INSTR_INIT, [0])

      yield self.await_program(mem,await_done=True,await_fail=True)
      base = bernoulli.rvs(0.5)
      if base < 0.5:
        mem.execute_instruction(instr.INSTR_H, [0])
        base = "plusmoins"
      else:
        mem.execute_instruction(instr.INSTR_I, [0])
        base = "zeroun"
      yield self.await_program(mem,await_done=True,await_fail=True)

      bit = bernoulli.rvs(0.5)
      if bit < 0.5:
        mem.execute_instruction(instr.INSTR_I, [0], physical=False)
      else:
        if base == "zeroun":
          mem.execute_instruction(instr.INSTR_X, [0], physical=False)
        elif base == "plusmoins":
          mem.execute_instruction(instr.INSTR_Z, [0], physical=False)
      qubit, = mem.pop([0])

In [ ]:
class ReceiveProtocol(NodeProtocol):
  # Protocol performed by a node to receive a state a measure it
  def __init__(self, othernode, port, node):
    super().__init__(node=node)
    self._othernode = othernode
    self._port = port
  def run(self):
    if self.node.name[0:9]== 'Satellite':
      mem = self.node.subcomponents["SatelliteMemory"]
    else:
      mem = self.node.subcomponents["StationMemory"]
      while True:
        yield self.await_port_input(self._port)
        base = bernoulli.rvs(0.5)
        if base < 0.5:
          mem.execute_instruction(instr.INSTR_H, [0], physical = False)
          base = "plusmoins"
        else:
          mem.execute_instruction(instr.INSTR_I, [0],physical = False)
          base = "zeroun"
        m,_,_ = mem.execute_instruction(instr.INSTR_MEASURE,[0],output_key="M1")
        yield self.await_program(mem,await_done=True,await_fail=True)
        Key.append(m['M1'][0])
        mem.reset()

In [ ]:
!pip install skyfield

In [ ]:
import skyfield.api
from skyfield.api import load, wgs84, Loader
from skyfield.api import Topos, utc
from skyfield.sgp4lib import EarthSatellite
from skyfield.constants import AU_KM, DAY_S
from skyfield.functions import length_of
from skyfield.functions import length_of
from skyfield.positionlib import ICRF
from datetime import datetime, timedelta

In [ ]:
class DownChannel:
  def __init__(self, tle, lat, lon, alt, wl=None, atmModel=None):
    self.latitude = lat
    self.longitude = lon
    self.altitude = alt
    self.tle = tle

    #self.wavelength = float(wl)

  def calculateParameters(self, start, end, step):

    ts = load.timescale(builtin=True)
    timelist = []
    num_tries = 10
    name, L1, L2 = self.tle.splitlines()
    sat = EarthSatellite(L1, L2)

    while start < end:
      timelist.append(start)
      start += step

    distance = np.zeros( (len(timelist),) )
    elevation = np.zeros( (len(timelist),) )

    for i in range(len(timelist)):

      t = ts.utc(timelist[i].year, timelist[i].month,
      timelist[i].day,timelist[i].hour,
      timelist[i].minute,timelist[i].second)

      ground_station = wgs84.latlon(self.latitude, self.longitude, self.altitude)
      diff = sat - ground_station

      alt, az, dist = diff.at(t).altaz()

      elevation[i] = alt.degrees
      distance[i] = dist.km

    return distance, elevation

In [ ]:
tle_AMAZONIA_1 = '''
1 47699U 21015A   24331.66896108  .00000823  00000+0  27046-3 0  9995
2 47699  98.3819  44.8584 0001784 110.4232 249.7148 14.40820713196923
'''
tle_STARLINK_2071 = """
1 48656U 21044U   24339.55275590  .00026086  00000+0  17641-2 0  9991
2 48656  53.0534  27.2526 0001973  90.0742 270.0473 15.06391245194358
"""
tle_IRIDIUM_122 = """
1 42957U 17061C   25007.49702594  .00000117  00000+0  34556-4 0  9991
2 42957  86.3957 244.8288 0001777  86.7983 273.3416 14.34214161379435
"""
tle_STARLINK_1008 = '''
1 44714U 19074B   25012.00402021  .00004094  00000+0  29358-3 0  9996
2 44714  53.0539 279.6404 0001242  80.1545 279.9584 15.06395110285360
'''

In [ ]:
start_iridium = datetime(2024, 2, 16, 6, 40, 0, tzinfo=utc)
end_iridium = datetime(2024, 2, 16, 7, 20, 0, tzinfo=utc)

start_starlink = datetime(2023, 2, 12, 9, 0, 0, tzinfo=utc)
end_starlink = datetime(2023, 2, 12, 10, 0, 0, tzinfo=utc)

start_amazon = datetime(2024, 10, 22, 13, 0, 0, tzinfo=utc)
end_amazon = datetime(2024, 10, 22, 14, 20, 0, tzinfo=utc)

start_starlink_1008 = datetime(2024, 10, 25, 7, 30, 0, tzinfo=utc)
end_starlink_1008 = datetime(2024, 10, 25, 8, 30, 0, tzinfo=utc)

Step = timedelta(seconds = 1.)

In [ ]:
lat_unicamp = -22.8542
lon_unicamp = -47.0220
alt_unicamp = 0.

In [ ]:
down_channel = DownChannel(tle_IRIDIUM_122, lat_unicamp, lon_unicamp, alt_unicamp)
distance, elevation = down_channel.calculateParameters(start_iridium, end_iridium, Step)
results = np.zeros((100,))

## Simulate QKD trough `FixedSatelliteLoss` Model and Free-Space Channel